In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline
import evaluate
from datasets import Dataset
from transformers import TrainerCallback
from copy import deepcopy

2023-04-21 16:25:23.937706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 16:25:23.994345: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-21 16:25:23.996152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-21 16:25:23.996159: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
dataset = pd.read_csv("DB/DB_hyperfiltered.csv") #read the dataset

dataset["Sequence"] = dataset["Sequence"].apply(lambda x: ' '.join(list(x))) #function to add spaces between each aa
dataset.rename(columns={'Sequence': 'text', 'Class': 'label'}, inplace=True) #rename the dataset colums to appropriate names for BERT to read

train_data = dataset.sample(frac=0.7) #split train and test sets
test_data = dataset.drop(train_data.index) 
hg_train_data = Dataset.from_pandas(train_data) #convert pandas datasets into huggingface datasets
hg_test_data = Dataset.from_pandas(test_data)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", model_max_length=200) #tokenize the inputs and set max length to 200(100 aa+100 spaces)
def tokenize_dataset(data):
    return tokenizer(data["text"], max_length=100, truncation=True, padding="max_length")
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1199/1199 [00:00<00:00, 13537.63ex/s]


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert", num_labels=2) #load pre-trained model
training_args = TrainingArguments(output_dir="NLP_files", logging_dir='NLP_files', logging_strategy='epoch',
    logging_steps=100, num_train_epochs=20, per_device_train_batch_size=4,  
    per_device_eval_batch_size=4, learning_rate=5e-6, save_strategy='epoch', save_steps=100,
    evaluation_strategy='epoch', eval_steps=100, load_best_model_at_end=True) 

def compute_metrics(eval_pred): #metrics to calculate
    metric1 = evaluate.load("matthews_correlation")
    metric2 = evaluate.load("roc_auc")
    metric3 = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    mcc = metric1.compute(predictions=predictions, references=labels)
    acc = metric3.compute(predictions=predictions, references=labels)
    pre = metric2.compute(prediction_scores=predictions, references=labels)
    
    return {"ROCAUC": pre, "MCC": mcc, "Accuracy": acc}

class CustomCallback(TrainerCallback): #custom callback to record training parameters 
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

trainer = Trainer(model=model, args=training_args, train_dataset=dataset_train, #training of the model
    eval_dataset=dataset_test, compute_metrics=compute_metrics)
trainer.add_callback(CustomCallback(trainer))

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

Epoch,Training Loss,Validation Loss,Rocauc,Mcc,Accuracy
1,No log,0.185029,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9621293319042515}
1,0.205200,0.140990,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9716430358632193}
2,0.205200,0.184353,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9621293319042515}
2,0.183000,0.140553,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9716430358632193}
3,0.183000,0.107838,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9621293319042515}
3,0.146600,0.095904,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9716430358632193}
4,0.146600,0.138826,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9621293319042515}
4,0.099900,0.120360,{'roc_auc': 0.5},{'matthews_correlation': 0.0},{'accuracy': 0.9716430358632193}
5,0.099900,0.081054,{'roc_auc': 0.6460407485514507},{'matthews_correlation': 0.524439731765786},{'accuracy': 0.9728474455162558}
5,0.100500,0.099696,{'roc_auc': 0.6029411764705883},{'matthews_correlation': 0.44857430865236264},{'accuracy': 0.9774812343619683}


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2799
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.5}" of type <class 'dict'> for key "train/train_ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.0}" of type <class 'dict'> for key "train/train_MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9621293319042515}" of type <class 'dict'> for key "train/train_Accuracy" as a scalar. This invocation of Ten

***** Running Evaluation *****
  Num examples = 1199
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.5}" of type <class 'dict'> for key "eval/ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.0}" of type <class 'dict'> for key "eval/MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9716430358632193}" of type <class 'dict'> for key "eval/Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to NLP_files/checkpoint-2800
Configuration saved in NLP_files/checkpoint-2800/config.json
Model weights saved in NLP_files/checkpoint-2800/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in

***** Running Evaluation *****
  Num examples = 2799
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.9181403218687163}" of type <class 'dict'> for key "train/train_ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.8684560085930582}" of type <class 'dict'> for key "train/train_MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9907109682029296}" of type <class 'dict'> for key "train/train_Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Fun

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2799
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.9609644851431735}" of type <class 'dict'> for key "train/train_ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.9261367683591039}" of type <class 'dict'> for key "train/train_MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9946409431939979}" of type <class 'dict'> for key "train/train_Accuracy" as a 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1199
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.8209543044685684}" of type <class 'dict'> for key "eval/ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.7055343424627903}" of type <class 'dict'> for key "eval/MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9849874895746455}" of type <class 'dict'> for key "eval/Accuracy" as a scalar. This invocati

Configuration saved in NLP_files/checkpoint-4900/config.json
Model weights saved in NLP_files/checkpoint-4900/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2799
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.9609644851431735}" of type <class 'dict'> for key "train/train_ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.9261367683591039}" of type <class 'dict'> for key "train/train_MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Traine

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1199
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.8209543044685684}" of type <class 'dict'> for key "eval/ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.7055343424627903}" of type <class 'dict'> for key "eval/MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9849874895746455}" of type <class 'dict'> for key "eval/Accuracy" as a scalar. This invocati

Configuration saved in NLP_files/checkpoint-9800/config.json
Model weights saved in NLP_files/checkpoint-9800/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, Function. If text, __index_level_0__, Function are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2799
  Batch size = 4
Trainer is attempting to log a value of "{'roc_auc': 0.9609644851431735}" of type <class 'dict'> for key "train/train_ROCAUC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'matthews_correlation': 0.9261367683591039}" of type <class 'dict'> for key "train/train_MCC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Traine